# 03. Gold Layer: 有病率・治療パターン分析

## 概要
Silverデータを用いて論文の主要結果を再現します。

## 分析内容
1. **有病率の推定** (Table 1相当)
2. **年齢層別有病率と性別比** (Table 2相当)
3. **年齢層別薬剤使用パターン** (Table 3相当)
4. **手術・検査実施率** (Table 4相当)
5. **結果の可視化と論文との比較**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'DejaVu Sans'
import os

# データディレクトリ
BRONZE_DIR = "../data/bronze"
SILVER_DIR = "../data/silver"
GOLD_DIR = "../data/gold"
os.makedirs(GOLD_DIR, exist_ok=True)

print("Gold Layer分析を開始します...")

## 1. データの読み込み

In [ ]:
# Silverデータの読み込み
df_ra_master = pd.read_parquet(f"{SILVER_DIR}/ra_patients_def3.parquet")
df_definitions = pd.read_parquet(f"{SILVER_DIR}/ra_definitions_summary.parquet")

# Bronzeデータ（総人口用）
df_patients = pd.read_parquet(f"{BRONZE_DIR}/patients.parquet")

print(f"RA患者数 (Definition 3): {len(df_ra_master):,}")
print(f"総患者数: {len(df_patients):,}")

## 2. Table 1相当: RA定義別有病率

In [ ]:
# 論文のTable 1の値（参考）
paper_table1 = {
    'Definition 0': {'n_patients': 1116122, 'prevalence': 0.88},
    'Definition 2': {'n_patients': 869340, 'prevalence': 0.69},
    'Definition 3': {'n_patients': 825772, 'prevalence': 0.65},
    'Definition 4': {'n_patients': 583137, 'prevalence': 0.46}
}

# 再現データの有病率
total_population = len(df_patients)

table1_data = []
for _, row in df_definitions.iterrows():
    def_name = row['definition'].replace('def_', 'Definition ')
    n_patients = row['n_patients']
    prevalence = row['prevalence_pct']
    
    # 論文の値
    paper_key = def_name.replace('Definition ', 'Definition ')
    paper_val = paper_table1.get(paper_key, {'n_patients': 'N/A', 'prevalence': 'N/A'})
    
    table1_data.append({
        'Definition': def_name,
        'N_patients (Reprod)': n_patients,
        'Prevalence % (Reprod)': f"{prevalence:.2f}",
        'Prevalence % (Paper)': paper_val['prevalence']
    })

df_table1 = pd.DataFrame(table1_data)
print("=" * 70)
print("Table 1: RA定義別の患者数と有病率")
print("=" * 70)
print(df_table1.to_string(index=False))
print("\n注: 論文は日本人口約1.27億人を分母、再現は{:,}人を分母".format(total_population))

## 3. Table 2相当: 年齢層別有病率と性別比

In [ ]:
# 年齢群の順序
age_group_order = ["16-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-84", "85+"]

# 論文Table 2の値（参考）
paper_table2 = {
    "16-19": {"n": 4389, "pct": 0.5, "female_pct": 64.0, "fm_ratio": 1.77, "prevalence": 0.03},
    "20-29": {"n": 12253, "pct": 1.5, "female_pct": 79.8, "fm_ratio": 3.94, "prevalence": 0.07},
    "30-39": {"n": 39519, "pct": 4.8, "female_pct": 80.7, "fm_ratio": 3.61, "prevalence": 0.27},
    "40-49": {"n": 83519, "pct": 10.1, "female_pct": 80.7, "fm_ratio": 4.18, "prevalence": 0.44},
    "50-59": {"n": 123201, "pct": 14.9, "female_pct": 79.3, "fm_ratio": 3.84, "prevalence": 0.79},
    "60-69": {"n": 218326, "pct": 26.4, "female_pct": 76.9, "fm_ratio": 3.33, "prevalence": 1.23},
    "70-79": {"n": 236137, "pct": 28.6, "female_pct": 74.3, "fm_ratio": 2.89, "prevalence": 1.63},
    "80-84": {"n": 50558, "pct": 6.1, "female_pct": 74.2, "fm_ratio": 2.88, "prevalence": 1.14},
    "85+": {"n": 57870, "pct": 7.0, "female_pct": 77.7, "fm_ratio": 3.49, "prevalence": 0.89}
}

def calculate_table2(df_ra: pd.DataFrame, df_all: pd.DataFrame) -> pd.DataFrame:
    """
    年齢層別の統計を計算
    """
    results = []
    
    for age_group in age_group_order:
        # RA患者
        ra_group = df_ra[df_ra['age_group'] == age_group]
        n_ra = len(ra_group)
        pct_ra = n_ra / len(df_ra) * 100 if len(df_ra) > 0 else 0
        
        # 性別
        n_female = (ra_group['sex'] == '2').sum()
        n_male = (ra_group['sex'] == '1').sum()
        female_pct = n_female / n_ra * 100 if n_ra > 0 else 0
        fm_ratio = n_female / n_male if n_male > 0 else float('inf')
        
        # 総人口（同年齢群）
        all_group = df_all[df_all['age_group'] == age_group]
        n_all = len(all_group)
        
        # 有病率
        prevalence = n_ra / n_all * 100 if n_all > 0 else 0
        
        # 論文の値
        paper = paper_table2.get(age_group, {})
        
        results.append({
            'Age Group': age_group,
            'N': n_ra,
            '% of Total': f"{pct_ra:.1f}",
            'Female %': f"{female_pct:.1f}",
            'F/M Ratio': f"{fm_ratio:.2f}",
            'Prevalence %': f"{prevalence:.2f}",
            '(Paper) %': paper.get('pct', 'N/A'),
            '(Paper) Prevalence': paper.get('prevalence', 'N/A')
        })
    
    # 合計行
    total_ra = len(df_ra)
    total_female = (df_ra['sex'] == '2').sum()
    total_male = (df_ra['sex'] == '1').sum()
    
    results.append({
        'Age Group': 'Total',
        'N': total_ra,
        '% of Total': '100.0',
        'Female %': f"{total_female/total_ra*100:.1f}",
        'F/M Ratio': f"{total_female/total_male:.2f}" if total_male > 0 else 'N/A',
        'Prevalence %': f"{total_ra/len(df_all)*100:.2f}",
        '(Paper) %': '100.0',
        '(Paper) Prevalence': '0.65'
    })
    
    return pd.DataFrame(results)

df_table2 = calculate_table2(df_ra_master, df_patients)

print("=" * 90)
print("Table 2: 年齢層別RA患者数、性別比、有病率")
print("=" * 90)
print(df_table2.to_string(index=False))

## 4. Table 3相当: 年齢層別薬剤使用パターン

In [ ]:
# 論文Table 3の値（参考、一部抜粋）
paper_table3_mtx = {
    "16-19": 59.9, "20-29": 60.9, "30-39": 64.7, "40-49": 69.9,
    "50-59": 73.1, "60-69": 70.9, "70-79": 60.4, "80-84": 50.5, "85+": 38.2
}

paper_table3_bdmard = {
    "16-19": 50.9, "20-29": 42.9, "30-39": 34.8, "40-49": 30.9,
    "50-59": 27.9, "60-69": 22.4, "70-79": 17.8, "80-84": 15.0, "85+": 13.7
}

def calculate_table3(df_ra: pd.DataFrame) -> pd.DataFrame:
    """
    年齢層別の薬剤使用率を計算
    """
    drug_cols = ['MTX', 'SSZ', 'BUC', 'TAC', 'IGT', 'LEF', 'TNFI', 'IL6I', 'ABT', 'JAKi', 'CS', 'bDMARDs']
    
    results = []
    
    for age_group in age_group_order:
        ra_group = df_ra[df_ra['age_group'] == age_group]
        n = len(ra_group)
        
        row = {'Age Group': age_group, 'N': n}
        
        for col in drug_cols:
            if col in ra_group.columns:
                usage = ra_group[col].mean() * 100 if n > 0 else 0
                row[col] = f"{usage:.1f}"
            else:
                row[col] = "N/A"
        
        # TNFI/ABT比率
        if 'TNFI' in ra_group.columns and 'ABT' in ra_group.columns:
            tnfi_rate = ra_group['TNFI'].mean()
            abt_rate = ra_group['ABT'].mean()
            ratio = tnfi_rate / abt_rate if abt_rate > 0 else float('inf')
            row['TNFI/ABT'] = f"{ratio:.1f}" if ratio != float('inf') else "N/A"
        
        results.append(row)
    
    # 合計行
    total_row = {'Age Group': 'Total', 'N': len(df_ra)}
    for col in drug_cols:
        if col in df_ra.columns:
            usage = df_ra[col].mean() * 100
            total_row[col] = f"{usage:.1f}"
        else:
            total_row[col] = "N/A"
    
    if 'TNFI' in df_ra.columns and 'ABT' in df_ra.columns:
        tnfi_rate = df_ra['TNFI'].mean()
        abt_rate = df_ra['ABT'].mean()
        ratio = tnfi_rate / abt_rate if abt_rate > 0 else float('inf')
        total_row['TNFI/ABT'] = f"{ratio:.1f}" if ratio != float('inf') else "N/A"
    
    results.append(total_row)
    
    return pd.DataFrame(results)

df_table3 = calculate_table3(df_ra_master)

print("=" * 120)
print("Table 3: 年齢層別薬剤使用率 (%)")
print("=" * 120)
# 主要な列のみ表示
display_cols = ['Age Group', 'N', 'MTX', 'SSZ', 'BUC', 'TNFI', 'IL6I', 'ABT', 'JAKi', 'CS', 'bDMARDs', 'TNFI/ABT']
print(df_table3[display_cols].to_string(index=False))

print("\n【論文との比較（MTX使用率）】")
for age_group in age_group_order:
    reprod = df_table3[df_table3['Age Group'] == age_group]['MTX'].values[0]
    paper = paper_table3_mtx.get(age_group, 'N/A')
    print(f"  {age_group}: 再現={reprod}%, 論文={paper}%")

## 5. Table 4相当: 手術・検査実施率

In [ ]:
# 論文Table 4の値（参考）
paper_table4 = {
    'TJR': 0.93,           # 人工関節置換術
    'ARTHROPLASTY': 0.32,  # 関節形成術
    'SYNOVECTOMY': 0.13,   # 滑膜切除術
    'Total_Surgery': 1.35  # RA関連手術合計
}

def calculate_table4(df_ra: pd.DataFrame) -> pd.DataFrame:
    """
    年齢層別の手術・検査実施率を計算
    """
    proc_cols = ['TJR', 'ARTHROPLASTY', 'SYNOVECTOMY', 'ULTRASOUND', 'BMD', 'any_RA_surgery']
    
    results = []
    
    for age_group in age_group_order:
        ra_group = df_ra[df_ra['age_group'] == age_group]
        n = len(ra_group)
        
        row = {'Age Group': age_group, 'N': n}
        
        for col in proc_cols:
            if col in ra_group.columns:
                usage = ra_group[col].mean() * 100 if n > 0 else 0
                row[col] = f"{usage:.2f}"
            else:
                row[col] = "N/A"
        
        results.append(row)
    
    # 合計行
    total_row = {'Age Group': 'Total', 'N': len(df_ra)}
    for col in proc_cols:
        if col in df_ra.columns:
            usage = df_ra[col].mean() * 100
            total_row[col] = f"{usage:.2f}"
        else:
            total_row[col] = "N/A"
    
    results.append(total_row)
    
    return pd.DataFrame(results)

df_table4 = calculate_table4(df_ra_master)

print("=" * 100)
print("Table 4: 年齢層別手術・検査実施率 (%)")
print("=" * 100)
print(df_table4.to_string(index=False))

print("\n【論文との比較（全年齢）】")
for proc, paper_val in paper_table4.items():
    if proc in df_table4.columns:
        reprod_val = df_table4[df_table4['Age Group'] == 'Total'][proc].values[0]
        print(f"  {proc}: 再現={reprod_val}%, 論文={paper_val}%")

## 6. 結果の可視化

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 年齢層別RA患者分布
ax1 = axes[0, 0]
age_counts = df_ra_master['age_group'].value_counts().reindex(age_group_order)
bars1 = ax1.bar(range(len(age_group_order)), age_counts.values, color='steelblue', alpha=0.7)
ax1.set_xticks(range(len(age_group_order)))
ax1.set_xticklabels(age_group_order, rotation=45)
ax1.set_xlabel('Age Group')
ax1.set_ylabel('Number of RA Patients')
ax1.set_title('Age Distribution of RA Patients (Definition 3)')
ax1.grid(axis='y', alpha=0.3)

# 2. 年齢層別MTX使用率（論文との比較）
ax2 = axes[0, 1]
mtx_reprod = []
mtx_paper = []
for ag in age_group_order:
    group = df_ra_master[df_ra_master['age_group'] == ag]
    mtx_reprod.append(group['MTX'].mean() * 100 if len(group) > 0 else 0)
    mtx_paper.append(paper_table3_mtx.get(ag, 0))

x = np.arange(len(age_group_order))
width = 0.35
ax2.bar(x - width/2, mtx_reprod, width, label='Reproduced', color='steelblue', alpha=0.7)
ax2.bar(x + width/2, mtx_paper, width, label='Paper', color='coral', alpha=0.7)
ax2.set_xticks(x)
ax2.set_xticklabels(age_group_order, rotation=45)
ax2.set_xlabel('Age Group')
ax2.set_ylabel('MTX Usage Rate (%)')
ax2.set_title('MTX Usage by Age Group')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. 年齢層別bDMARDs使用率
ax3 = axes[1, 0]
bdmard_reprod = []
bdmard_paper = []
for ag in age_group_order:
    group = df_ra_master[df_ra_master['age_group'] == ag]
    bdmard_reprod.append(group['bDMARDs'].mean() * 100 if len(group) > 0 else 0)
    bdmard_paper.append(paper_table3_bdmard.get(ag, 0))

ax3.bar(x - width/2, bdmard_reprod, width, label='Reproduced', color='steelblue', alpha=0.7)
ax3.bar(x + width/2, bdmard_paper, width, label='Paper', color='coral', alpha=0.7)
ax3.set_xticks(x)
ax3.set_xticklabels(age_group_order, rotation=45)
ax3.set_xlabel('Age Group')
ax3.set_ylabel('bDMARDs Usage Rate (%)')
ax3.set_title('bDMARDs Usage by Age Group')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. TNFI/ABT比率の年齢変化
ax4 = axes[1, 1]
tnfi_abt_ratio = []
for ag in age_group_order:
    group = df_ra_master[df_ra_master['age_group'] == ag]
    if len(group) > 0:
        tnfi = group['TNFI'].mean()
        abt = group['ABT'].mean()
        ratio = tnfi / abt if abt > 0 else 0
        tnfi_abt_ratio.append(ratio)
    else:
        tnfi_abt_ratio.append(0)

ax4.plot(age_group_order, tnfi_abt_ratio, 'o-', color='darkgreen', linewidth=2, markersize=8)
ax4.set_xlabel('Age Group')
ax4.set_ylabel('TNFI/ABT Ratio')
ax4.set_title('TNFI to ABT Usage Ratio by Age')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f"{GOLD_DIR}/analysis_results.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n図を保存しました: {GOLD_DIR}/analysis_results.png")

## 7. 主要結果のサマリー

In [ ]:
# 主要指標の計算
total_ra = len(df_ra_master)
total_pop = len(df_patients)
prevalence = total_ra / total_pop * 100

female_ratio = (df_ra_master['sex'] == '2').mean() * 100
elderly_65_ratio = (df_ra_master['age'] >= 65).mean() * 100
elderly_85_ratio = (df_ra_master['age'] >= 85).mean() * 100

mtx_rate = df_ra_master['MTX'].mean() * 100
bdmard_rate = df_ra_master['bDMARDs'].mean() * 100
cs_rate = df_ra_master['CS'].mean() * 100

surgery_rate = df_ra_master['any_RA_surgery'].mean() * 100

# サマリーテーブル
summary_data = {
    'Metric': [
        'Total RA Patients (Definition 3)',
        'Prevalence (%)',
        'Female Ratio (%)',
        'Age >= 65 years (%)',
        'Age >= 85 years (%)',
        'MTX Usage (%)',
        'bDMARDs Usage (%)',
        'Corticosteroid Usage (%)',
        'RA Surgery Rate (%)'
    ],
    'Reproduced': [
        f"{total_ra:,}",
        f"{prevalence:.2f}",
        f"{female_ratio:.1f}",
        f"{elderly_65_ratio:.1f}",
        f"{elderly_85_ratio:.1f}",
        f"{mtx_rate:.1f}",
        f"{bdmard_rate:.1f}",
        f"{cs_rate:.1f}",
        f"{surgery_rate:.2f}"
    ],
    'Paper': [
        '825,772',
        '0.65',
        '76.3',
        '60.8',
        '7.0',
        '63.4',
        '22.9',
        '~45',
        '1.35'
    ]
}

df_summary = pd.DataFrame(summary_data)

print("=" * 70)
print("主要結果サマリー: 再現 vs 論文")
print("=" * 70)
print(df_summary.to_string(index=False))

## 8. Goldデータの保存

In [ ]:
# 結果テーブルの保存
df_table1.to_parquet(f"{GOLD_DIR}/table1_definitions.parquet", index=False)
df_table2.to_parquet(f"{GOLD_DIR}/table2_age_distribution.parquet", index=False)
df_table3.to_parquet(f"{GOLD_DIR}/table3_medication.parquet", index=False)
df_table4.to_parquet(f"{GOLD_DIR}/table4_procedures.parquet", index=False)
df_summary.to_parquet(f"{GOLD_DIR}/summary.parquet", index=False)

# CSVでも保存（確認用）
df_table1.to_csv(f"{GOLD_DIR}/table1_definitions.csv", index=False)
df_table2.to_csv(f"{GOLD_DIR}/table2_age_distribution.csv", index=False)
df_table3.to_csv(f"{GOLD_DIR}/table3_medication.csv", index=False)
df_table4.to_csv(f"{GOLD_DIR}/table4_procedures.csv", index=False)
df_summary.to_csv(f"{GOLD_DIR}/summary.csv", index=False)

print("Goldデータを保存しました:")
print(f"  - table1_definitions.parquet/csv")
print(f"  - table2_age_distribution.parquet/csv")
print(f"  - table3_medication.parquet/csv")
print(f"  - table4_procedures.parquet/csv")
print(f"  - summary.parquet/csv")
print(f"  - analysis_results.png")

## 9. 論文の主要な発見との対応

In [ ]:
print("=" * 70)
print("論文の主要な発見と再現結果の対応")
print("=" * 70)

findings = [
    {
        "finding": "1. RA有病率は0.65%",
        "paper": "825,772人、有病率0.65%",
        "reproduced": f"{total_ra:,}人、有病率{prevalence:.2f}%"
    },
    {
        "finding": "2. 女性が76.3%を占める",
        "paper": "76.3%",
        "reproduced": f"{female_ratio:.1f}%"
    },
    {
        "finding": "3. 65歳以上が60.8%",
        "paper": "60.8%",
        "reproduced": f"{elderly_65_ratio:.1f}%"
    },
    {
        "finding": "4. 70-79歳群で最高の有病率1.63%",
        "paper": "1.63%",
        "reproduced": "(計算済み - Table 2参照)"
    },
    {
        "finding": "5. MTX使用率は年齢とともに減少",
        "paper": "40-49歳: 69.9% -> 85歳以上: 38.2%",
        "reproduced": "(グラフで確認可能)"
    },
    {
        "finding": "6. bDMARDs使用率は若年で高い",
        "paper": "16-19歳: 50.9% -> 85歳以上: 13.7%",
        "reproduced": "(グラフで確認可能)"
    },
    {
        "finding": "7. TNFI/ABT比は年齢とともに減少",
        "paper": "16-19歳: 24.0:1 -> 85歳以上: 1.7:1",
        "reproduced": "(グラフで確認可能)"
    }
]

for f in findings:
    print(f"\n{f['finding']}")
    print(f"  論文: {f['paper']}")
    print(f"  再現: {f['reproduced']}")

print("\n" + "=" * 70)
print("Gold Layer 分析完了")
print("=" * 70)

## 10. 勘所のまとめ

### このノートブックで学べること

#### データフローの勘所
1. **Bronze -> Silver**: 生データに対して「定義」を適用して分析対象を絞り込む
   - ICD-10コードによる疾患同定
   - 処方月数による患者定義の精緻化

2. **Silver -> Gold**: 分析単位での集計と指標算出
   - 年齢群別の集計
   - 使用率・実施率の計算

#### NDBデータ解析の勘所
1. **患者同定**: 診断コード（ICD-10）だけでなく、治療薬処方も組み合わせる
2. **処方月数**: 「2ヶ月以上」などの閾値で患者定義を厳格化
3. **年齢層別分析**: 高齢者の治療パターンは若年者と異なる
4. **性別比**: RAは女性に多い（F/M比で評価）

#### 論文再現のポイント
1. 論文の「定義」を正確に理解・実装する
2. 複数の定義を比較して妥当性を確認
3. 結果を論文の表と照合して検証